In [ ]:
from transformers import pipeline
from datasets import load_dataset
from tqdm import tqdm

model_name = "distilbert/distilbert-base-cased-distilled-squad"
qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

from datasets import load_dataset

dataset = load_dataset("lucadiliello/newsqa")

In [ ]:
from tqdm import tqdm

predictions = []
references = []

validation_dataset = dataset['validation']

sample_dataset = validation_dataset.select(range(500))

print(f"Running evaluation on {len(sample_dataset)} examples...")

for i, example in enumerate(tqdm(sample_dataset)):

    question = example["question"]
    context = example["context"]
    result = qa_pipeline(question=question, context=context)
    prediction_text = result['answer']

    pred_obj = {
        'id': str(i),
        'prediction_text': prediction_text
    }
    predictions.append(pred_obj)


    ground_truth_answers = example['answers']
    answer_starts = []

    for answer_text in ground_truth_answers:
        start_index = context.find(answer_text)
        if start_index != -1:
            answer_starts.append(start_index)
        else:
            answer_starts.append(0)

    ref_obj = {
        'id': str(i),
        'answers': {
            'text': ground_truth_answers,
            'answer_start': answer_starts
        }
    }
    references.append(ref_obj)

In [ ]:
!pip install evaluate
import evaluate
squad_metric = evaluate.load("squad")
results = squad_metric.compute(predictions=predictions, references=references)

print(f"Total Examples:   {len(predictions)}")
print(f"Exact Match (EM): {results['exact_match']:.2f}")
print(f"F1 Score (F1):    {results['f1']:.2f}")